In [1]:
import pandas as pd
import cufflinks as cf
import warnings
warnings.filterwarnings("ignore")
from plotly.offline import plot,iplot
pd.options.plotting.backend = "plotly"
import plotly.graph_objects as go
import plotly.express as px#graficos express
cf.go_offline()
pd.set_option("display.max_columns",200)

## FUNCIONES

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
def rename_cols(df,cols,prefix):
    new_feats=[prefix+col for col in cols]
    df=df.rename(columns=dict(zip(cols,new_feats)))
    return df

def bar(df,col,title,x_title="",y_title=""):
    layout = go.Layout(font_family="Courier New, monospace",
    font_color="black",title_text=title,title_font_size=30,xaxis= {"title": {"text": x_title,"font": {"family": 'Courier New, monospace',"size": 18,
        "color": '#002e4d'}}},yaxis= {"title": {"text": y_title,"font": {"family": 'Courier New, monospace',"size": 18,
        "color": '#002e4d'}}},title_font_family="Courier New, monospace",title_font_color="#004878",template="plotly_white")
    aux=pd.DataFrame(df[col].value_counts()).reset_index().rename(columns={"index":"conteo"})
    fig=aux.iplot(kind='bar',x="conteo",y=col,title=title,asFigure=True,barmode="overlay",sortbars=True,color="#255479",layout=layout)
    fig.update_layout(width=800)
    fig.update_traces(marker_color='#005a96')
    return fig

def pie(df,col,title,x_title="",y_title=""):
    layout = go.Layout(template="plotly_white")
    colors=[ "#152337", "#183152","#17416d","#005096","#00569c","#005ba3","#0061a9","#1567af","#226cb6","#2c72bc", "#0061a9","#4c79b7","#7492c6","#98acd4","#bbc7e2","#dde3f1","#ffffff"
]
    aux=pd.DataFrame(df[col].value_counts()).reset_index().rename(columns={"index":"conteo"})
    fig=aux.iplot(kind='pie',labels="conteo",values=col,title=title,asFigure=True,theme="white")

    fig.update_traces(textfont_size=10,
                  marker=dict(colors=colors, line=dict(color='#000000', width=2)))
    fig.update_traces(textposition='inside', textinfo='percent+label')
    fig.update_layout(font_family="Courier New, monospace",
    font_color="black",title_text=title,title_font_size=30,title_font_family="Courier New, monospace",title_font_color="#004878",template="plotly_white")
    return fig
def box(df,col,title):
    layout = go.Layout(font_family="Courier New, monospace",
    font_color="black",title_text=title,title_font_size=30,xaxis= {"title": {"font": {"family": 'Courier New, monospace',"size": 18,
        "color": '#002e4d'}}},title_font_family="Courier New, monospace",title_font_color="#004878",template="plotly_white")
    fig=df[[col]].iplot(kind='box',title=title,asFigure=True,theme="white",layout=layout,color="#005a96", boxpoints='outliers')
    return fig

def histogram(df,col,bins,title):
    layout = go.Layout(font_family="Courier New, monospace",
    font_color="black",title_text=title,title_font_size=30,xaxis= {"title": {"font": {"family": 'Courier New, monospace',"size": 18,
        "color": '#002e4d'}}},title_font_family="Courier New, monospace",title_font_color="#004878",template="plotly_white")
    fig=df[[col]].iplot(kind='histogram',x=col,bins=bins,title=title,asFigure=True,theme="white",layout=layout,color="#003e6c")
    fig.update_traces(opacity=0.90)
    return fig

def completitud(df):
    comp=pd.DataFrame(df.isnull().sum())
    comp.reset_index(inplace=True)
    comp=comp.rename(columns={"index":"columna",0:"total"})
    comp["completitud"]=(1-comp["total"]/df.shape[0])*100
    comp=comp.sort_values(by="completitud",ascending=True)
    comp.reset_index(drop=True,inplace=True)
    return comp

## DATOS

In [ ]:
df=pd.read_csv("../data/visualizaciones/dataset_house_prices.csv")

### DICCIONARIO DE DATOS

**El problema**

En análisis exploratorio tiene el objetivo de entender el significado y la relevancia de cada variable con respecto al problema. Además apartir de este análisis se pueden determinar que acciones tomar para cada variable En este caso:

- La relevancia de la variable en la compra de una casa.
- La importancia de la variable.
- Solapamiento con otras variables.


Variable objetivo : 'SalePrice'.

- MSSubClass: clase de construcción
- MSZoning: clasificación de la zona
- LotFrontage: pies lineales de calle de la parcela
- LotArea: tamaño de la parcela en pies cuadrados
- Street: tipo de acceso por carretera
- Alley: tipo de acceso al callejón
- LotShape: forma de la parcela
- LandContour: planitud de la parcela
- Utilities: servicios públicos disponibles
- LotConfig: Configuración de parcela
- LandSlope: pendiente de la parcela
- Neighborhood: ubicación física dentro de los límites de la ciudad de Ames
- Condition1: proximidad a la carretera principal o al ferrocarril
- Condition2: proximidad a la carretera principal o al ferrocarril (si hay un segundo)
- BldgType: tipo de vivienda
- HouseStyle: estilo de vivienda
- OverallQual: calidad general del material y del acabado
- OverallCond: condición general
- YearBuilt: fecha original de construcción
- YearRemodAdd: fecha de remodelación
- RoofStyle: tipo de cubierta
- RoofMatl: material del techo
- Exterior1st: revestimiento exterior de la casa
- Exterior2nd: revestimiento exterior de la casa (si hay más de un material)
- MasVnrType: tipo de revestimiento de mampostería
- MasVnrArea: área de revestimiento de mampostería en pies cuadrados
- ExterQual: calidad del material exterior
- ExterCond: estado del material en el exterior
- Foundation: tipo de cimentación
- BsmtQual: altura del sótano
- BsmtCond: estado general del sótano
- BsmtExposure: paredes del sótano a nivel de calle o de jardín
- BsmtFinType1: calidad del área acabada del sótano
- BsmtFinSF1: pies cuadrados de la superficie acabada tipo 1
- BsmtFinType2: calidad de la segunda superficie acabada (si existe)
- BsmtFinSF2: Pies cuadrados de la superficie acabada tipo 2
- BsmtUnfSF: pies cuadrados del área sin terminar del sótano
- TotalBsmtSF: pies cuadrados totales del sótano
- Heating: tipo de calefacción
- HeatingQC: calidad y estado de la calefacción
- CentralAir: aire acondicionado central
- Electrical: sistema eléctrico
- 1erFlrSF: área en pies cuadrados de la primera planta (o planta baja)
- 2ndFlrSF: área en pies cuadrados de la segunda planta
- LowQualFinSF: pies cuadrados acabados de baja calidad (todos los pisos)
- GrLivArea: superficie habitable por encima del nivel del suelo en pies cuadrados
- BsmtFullBath: cuartos de baño completos en el sótano
- BsmtHalfBath: medio baño del sótano
- FullBath: baños completos sobre el nivel del suelo
- HalfBath: medios baños sobre el nivel del suelo
- Bedroom: número de dormitorios por encima del nivel del sótano
- Kitchen: número de cocinas
- KitchenQual: calidad de la cocina
- TotRmsAbvGrd: total de habitaciones por encima del nivel del suelo (no incluye baños)
- Functional: valoración de la funcionalidad de la vivienda
- Fireplaces: número de chimeneas
- FireplaceQu: calidad de la chimenea
- GarageType: ubicación del garaje
- GarageYrBlt: año de construcción del garaje
- GarageFinish: acabado interior del garaje
- GarageCars: tamaño del garaje en capacidad de coches
- GarageArea: tamaño del garaje en pies cuadrados
- GarageQual: calidad de garaje
- GarageCond: condición de garaje
- PavedDrive: calzada asfaltada
- WoodDeckSF: area de plataforma de madera en pies cuadrados
- OpenPorchSF: área de porche abierto en pies cuadrados
- EnclosedPorch: área de porche cerrada en pies cuadrados
- 3SsnPorch: área de porche de tres estaciones en pies cuadrados
- ScreenPorch: superficie acristalada del porche en pies cuadrados
- PoolArea: área de la piscina en pies cuadrados
- PoolQC: calidad de la piscina
- Fence: calidad de la valla
- MiscFeature: característica miscelánea no cubierta en otras categorías
- MiscVal: valor en dólares de la característica miscelánea
- MoSold: mes de venta
- YrSold: año de venta
- SaleType: tipo de venta
- SaleCondition: Condiciones de venta

In [ ]:
df.shape

(1460, 81)

In [ ]:
for col in df.columns:
    print(col)
    display(df[col].value_counts())

Id


1       1
982     1
980     1
979     1
978     1
       ..
485     1
484     1
483     1
482     1
1460    1
Name: Id, Length: 1460, dtype: int64

MSSubClass


20     536
60     299
50     144
120     87
30      69
160     63
70      60
80      58
90      52
190     30
85      20
75      16
45      12
180     10
40       4
Name: MSSubClass, dtype: int64

MSZoning


...

WD       1267
New       122
COD        43
ConLD       9
ConLI       5
ConLw       5
CWD         4
Oth         3
Con         2
Name: SaleType, dtype: int64

SaleCondition


Normal     1198
Partial     125
Abnorml     101
Family       20
Alloca       12
AdjLand       4
Name: SaleCondition, dtype: int64

SalePrice


140000    20
135000    17
155000    14
145000    14
190000    13
          ..
202665     1
164900     1
208300     1
181500     1
147500     1
Name: SalePrice, Length: 663, dtype: int64

In [ ]:
v_feats=["MSSubClass","MSZoning","Street","Alley","LotShape","LandContour","Utilities","LotConfig","LandSlope",
         "Neighborhood","Condition1","Condition2","BldgType","HouseStyle","OverallQual","OverallCond",
        "YearBuilt","YearRemodAdd","RoofStyle","RoofMatl","Exterior1st","Exterior2nd","MasVnrType","ExterQual",
        "ExterCond","Foundation","BsmtQual","BsmtCond","BsmtExposure","BsmtFinType1","BsmtFinType2",
        "Heating","HeatingQC","CentralAir","Electrical","LowQualFinSF","BsmtHalfBath","FullBath","KitchenAbvGr","KitchenQual","TotRmsAbvGrd","Functional","Fireplaces","FireplaceQu",
        "GarageType","GarageYrBlt","GarageFinish","GarageQual","GarageCond","PavedDrive","PoolQC","Fence",
        "MiscFeature","MoSold","YrSold","SaleType","SaleCondition"]
c_feats=["LotFrontage","LotArea","MasVnrArea","BsmtFinSF1","BsmtFinSF2","BsmtUnfSF","TotalBsmtSF","1stFlrSF",
        "2ndFlrSF","GrLivArea","GarageArea","WoodDeckSF","OpenPorchSF","EnclosedPorch","3SsnPorch","ScreenPorch",
        "PoolArea","MiscVal","BedroomAbvGr","HalfBath","GarageCars","BsmtFullBath"]
tgt=["SalePrice"]

df=rename_cols(df,v_feats,"v_")
df=rename_cols(df,c_feats,"c_")
df=rename_cols(df,tgt,"tgt_")

## EDA

In [ ]:
for col in df.filter(like="v_"):
    bar(df,col,col).show()

In [ ]:
df.duplicated().sum()

0

### MISSINGS

In [ ]:
miss=completitud(df)
miss=miss[miss["total"]>0]

In [ ]:
miss

,columna,total,completitud
0,v_PoolQC,1453,0.479452
1,v_MiscFeature,1406,3.698630
2,v_Alley,1369,6.232877
3,v_Fence,1179,19.246575
4,v_FireplaceQu,690,52.739726
5,c_LotFrontage,259,82.260274
6,v_GarageYrBlt,81,94.452055
7,v_GarageType,81,94.452055
8,v_GarageFinish,81,94.452055
9,v_GarageQual,81,94.452055


### Nivel Nominal

- En este nivel, no podemos realizar ninguna operación matemática cuantitativa, como suma o división. No podemos encontrar un valor medio a nivel nominal.
- No hay un nombre promedio o un departamento de trabajo promedio.
- Sin embargo, podemos hacer recuentos básicos.

In [ ]:
df.filter(like="v_")

,v_MSSubClass,v_MSZoning,v_Street,v_Alley,v_LotShape,v_LandContour,v_Utilities,v_LotConfig,v_LandSlope,v_Neighborhood,v_Condition1,v_Condition2,v_BldgType,v_HouseStyle,v_OverallQual,v_OverallCond,v_YearBuilt,v_YearRemodAdd,v_RoofStyle,v_RoofMatl,v_Exterior1st,v_Exterior2nd,v_MasVnrType,v_ExterQual,v_ExterCond,v_Foundation,v_BsmtQual,v_BsmtCond,v_BsmtExposure,v_BsmtFinType1,v_BsmtFinType2,v_Heating,v_HeatingQC,v_CentralAir,v_Electrical,v_LowQualFinSF,v_BsmtHalfBath,v_FullBath,v_KitchenAbvGr,v_KitchenQual,v_TotRmsAbvGrd,v_Functional,v_Fireplaces,v_FireplaceQu,v_GarageType,v_GarageYrBlt,v_GarageFinish,v_GarageQual,v_GarageCond,v_PavedDrive,v_PoolQC,v_Fence,v_MiscFeature,v_MoSold,v_YrSold,v_SaleType,v_SaleCondition
0,60,RL,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,Gd,TA,PConc,Gd,TA,No,GLQ,Unf,GasA,Ex,Y,SBrkr,0,0,2,1,Gd,8,Typ,0,NaN,Attchd,2003.0,RFn,TA,TA,Y,NaN,NaN,NaN,2,2008,WD,Normal
1,20,RL,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,TA,TA,CBlock,Gd,TA,Gd,ALQ,Unf,GasA,Ex,Y,SBrkr,0,1,2,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,TA,TA,Y,NaN,NaN,NaN,5,2007,WD,Normal
2,60,RL,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,Gd,TA,PConc,Gd,TA,Mn,GLQ,Unf,GasA,Ex,Y,SBrkr,0,0,2,1,Gd,6,Typ,1,TA,Attchd,2001.0,RFn,TA,TA,Y,NaN,NaN,NaN,9,2008,WD,Normal
3,70,RL,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,TA,TA,BrkTil,TA,Gd,No,ALQ,Unf,GasA,Gd,Y,SBrkr,0,0,1,1,Gd,7,Typ,1,Gd,Detchd,1998.0,Unf,TA,TA,Y,NaN,NaN,NaN,2,2006,WD,Abnorml
4,60,RL,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,Gd,TA,PConc,Gd,TA,Av,GLQ,Unf,GasA,Ex,Y,SBrkr,0,0,2,1,Gd,9,Typ,1,TA,Attchd,2000.0,RFn,TA,TA,Y,NaN,NaN,NaN,12,2008,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,60,RL,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,6,5,1999,2000,Gable,CompShg,VinylSd,VinylSd,None,TA,TA,PConc,Gd,TA,No,Unf,Unf,GasA,Ex,Y,SBrkr,0,0,2,1,TA,7,Typ,1,TA,Attchd,1999.0,RFn,TA,TA,Y,NaN,NaN,NaN,8,2007,WD,Normal
1456,20,RL,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,NWAmes,Norm,Norm,1Fam,1Story,6,6,1978,1988,Gable,CompShg,Plywood,Plywood,Stone,TA,TA,CBlock,Gd,TA,No,ALQ,Rec,GasA,TA,Y,SBrkr,0,0,2,1,TA,7,Min1,2,TA,Attchd,1978.0,Unf,TA,TA,Y,NaN,MnPrv,NaN,2,2010,WD,Normal
1457,70,RL,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,9,1941,2006,Gable,CompShg,CemntBd,CmentBd,None,Ex,Gd,Stone,TA,Gd,No,GLQ,Unf,GasA,Ex,Y,SBrkr,0,0,2,1,Gd,9,Typ,2,Gd,Attchd,1941.0,RFn,TA,TA,Y,NaN,GdPrv,Shed,5,2010,WD,Normal
1458,20,RL,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Norm,Norm,1Fam,1Story,5,6,1950,1996,Hip,CompShg,MetalSd,MetalSd,None,TA,TA,CBlock,TA,TA,Mn,GLQ,Rec,GasA,Gd,Y,FuseA,0,0,1,1,Gd,5,Typ,0,NaN,Attchd,1950.0,Unf,TA,TA,Y,NaN,NaN,NaN,4,2010,WD,Normal


In [ ]:
street=pd.DataFrame(df["v_Street"].value_counts().reset_index()).rename(columns={"index":"categoria","v_Street":"valor"})

In [ ]:
street

,categoria,valor
0,Pave,1454
1,Grvl,6


<h1 style="color:green"> BARRAS </h1>

In [ ]:
bar(df,"v_Street","v_Street")

<h1 style="color:green"> PIE </h1>

In [ ]:
pie(df,"v_Street","")

In [ ]:
#TIPO DE ZONA
pd.DataFrame(df["v_MSZoning"].value_counts()).reset_index()

,index,v_MSZoning
0,RL,1151
1,RM,218
2,FV,65
3,RH,16
4,C (all),10


**MSZoning: Identifies the general zoning classification of the sale.**
- A Agriculture
- C Commercial
- FV Floating Village Residential
- I Industrial
- RH Residential High Density
- RL Residential Low Density
- RP Residential Low Density Park
- RM Residential Medium Density

In [ ]:
pd.DataFrame(df["v_MSZoning"].value_counts()).reset_index()

,index,v_MSZoning
0,RL,1151
1,RM,218
2,FV,65
3,RH,16
4,C (all),10


In [ ]:
pie(df,"v_MSZoning","v_MSZoning")

In [ ]:
df["v_MSZoning"]=df["v_MSZoning"].replace(dict(zip(list(df["v_MSZoning"].value_counts(1)[-3:].index),["otro"]*3)))

### Nivel Ordinal

- En el nivel ordinal, todavía podemos hacer recuentos básicos como lo hicimos en el nivel nominal, pero también podemos introducir comparaciones y ordenaciones en la mezcla. Por esta razón, podemos utilizar nuevos gráficos en este nivel. Podemos usar gráficos de barras y circulares como lo hicimos en el nivel nominal, pero debido a que ahora tenemos ordenamiento y comparaciones, podemos calcular medianas y percentiles. Con medianas y percentiles, son posibles los diagramas de caja.

**GarageQual: Garage quality**
- Ex Excellent
- Gd Good
- TA Typical/Average
- Fa Fair
- Po Poor
- NA No Garage

In [ ]:
dictio_garagequal=dict(zip(["Ex","Gd","TA","Fa","Po","NA"],[5,4,3,2,1,0]))

In [ ]:
df["v_GarageQual"].isnull().sum()

81

In [ ]:
df["v_GarageQual"].value_counts()

TA    1311
Fa      48
Gd      14
Ex       3
Po       3
Name: v_GarageQual, dtype: int64

In [ ]:
df["v_GarageQual"].fillna("NA").replace(dictio_garagequal).value_counts()

3    1311
0      81
2      48
4      14
5       3
1       3
Name: v_GarageQual, dtype: int64

In [ ]:
df["v_GarageQual"].fillna("NA")

0       TA
1       TA
2       TA
3       TA
4       TA
        ..
1455    TA
1456    TA
1457    TA
1458    TA
1459    TA
Name: v_GarageQual, Length: 1460, dtype: object

In [ ]:
#df["v_GarageQual"]=df["v_GarageQual"].fillna("NA").replace(dictio_garagequal)
df["v_ExterQual"]=df["v_ExterQual"].fillna("NA").replace(dictio_garagequal)

In [ ]:
garagequal=pd.DataFrame(df["v_GarageQual"].value_counts()).reset_index()

In [ ]:
garagequal

,index,v_GarageQual
0,TA,1311
1,Fa,48
2,Gd,14
3,Ex,3
4,Po,3


In [ ]:
box(df,"v_ExterQual","v_ExterQual")

In [ ]:
df[df["v_ExterQual"]==5]["v_GarageQual"].describe()

count     52
unique     2
top       TA
freq      51
Name: v_GarageQual, dtype: object

In [ ]:
df[['v_GarageQual', 'v_ExterQual']].pivot(columns='v_ExterQual', values='v_GarageQual').iplot(kind='box')

**MES DE VENTA**

In [ ]:
bar(df,"v_MoSold","_","v_MoSold")

In [ ]:
df["v_MoSold"].describe()

count    1460.000000
mean        6.321918
std         2.703626
min         1.000000
25%         5.000000
50%         6.000000
75%         8.000000
max        12.000000
Name: v_MoSold, dtype: float64

In [ ]:
box(df,"v_MoSold","v_MoSold")

In [ ]:
df["v_YrSold"].describe()

count    1460.000000
mean     2007.815753
std         1.328095
min      2006.000000
25%      2007.000000
50%      2008.000000
75%      2009.000000
max      2010.000000
Name: v_YrSold, dtype: float64

In [ ]:
box(df,"v_YrSold","v_YrSold")

### CONTINUAS
**Con la capacidad de sumar valores, podemos introducir dos conceptos familiares, la media aritmética (denominada simplemente media) y la desviación estándar.**
- Desviación estandar
- Percentiles
- Valor mínimo
- Valor maximo

In [ ]:
df.filter(like="c_")

,c_LotFrontage,c_LotArea,c_MasVnrArea,c_BsmtFinSF1,c_BsmtFinSF2,c_BsmtUnfSF,c_TotalBsmtSF,c_1stFlrSF,c_2ndFlrSF,c_GrLivArea,c_BsmtFullBath,c_HalfBath,c_BedroomAbvGr,c_GarageCars,c_GarageArea,c_WoodDeckSF,c_OpenPorchSF,c_EnclosedPorch,c_3SsnPorch,c_ScreenPorch,c_PoolArea,c_MiscVal
0,65.0,8450,196.0,706,0,150,856,856,854,1710,1,1,3,2,548,0,61,0,0,0,0,0
1,80.0,9600,0.0,978,0,284,1262,1262,0,1262,0,0,3,2,460,298,0,0,0,0,0,0
2,68.0,11250,162.0,486,0,434,920,920,866,1786,1,1,3,2,608,0,42,0,0,0,0,0
3,60.0,9550,0.0,216,0,540,756,961,756,1717,1,0,3,3,642,0,35,272,0,0,0,0
4,84.0,14260,350.0,655,0,490,1145,1145,1053,2198,1,1,4,3,836,192,84,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,62.0,7917,0.0,0,0,953,953,953,694,1647,0,1,3,2,460,0,40,0,0,0,0,0
1456,85.0,13175,119.0,790,163,589,1542,2073,0,2073,1,0,3,2,500,349,0,0,0,0,0,0
1457,66.0,9042,0.0,275,0,877,1152,1188,1152,2340,0,0,4,1,252,0,60,0,0,0,0,2500
1458,68.0,9717,0.0,49,1029,0,1078,1078,0,1078,1,0,2,1,240,366,0,112,0,0,0,0


In [ ]:
fig = go.Figure(data=[go.Histogram(y=df["c_LotArea"])])
fig.show()

In [ ]:
histogram(df[df["c_LotArea"]<=11601],"c_LotArea",12,"c_LotArea")

In [ ]:
df[df["v_ExterQual"]==5][["c_LotArea"]].iplot(kind="histogram",x="c_LotArea")

In [ ]:
df[df["c_LotArea"]<=11601]

,Id,v_MSSubClass,v_MSZoning,c_LotFrontage,c_LotArea,v_Street,v_Alley,v_LotShape,v_LandContour,v_Utilities,v_LotConfig,v_LandSlope,v_Neighborhood,v_Condition1,v_Condition2,v_BldgType,v_HouseStyle,v_OverallQual,v_OverallCond,v_YearBuilt,v_YearRemodAdd,v_RoofStyle,v_RoofMatl,v_Exterior1st,v_Exterior2nd,v_MasVnrType,c_MasVnrArea,v_ExterQual,v_ExterCond,v_Foundation,v_BsmtQual,v_BsmtCond,v_BsmtExposure,v_BsmtFinType1,c_BsmtFinSF1,v_BsmtFinType2,c_BsmtFinSF2,c_BsmtUnfSF,c_TotalBsmtSF,v_Heating,v_HeatingQC,v_CentralAir,v_Electrical,c_1stFlrSF,c_2ndFlrSF,v_LowQualFinSF,c_GrLivArea,c_BsmtFullBath,v_BsmtHalfBath,v_FullBath,c_HalfBath,c_BedroomAbvGr,v_KitchenAbvGr,v_KitchenQual,v_TotRmsAbvGrd,v_Functional,v_Fireplaces,v_FireplaceQu,v_GarageType,v_GarageYrBlt,v_GarageFinish,c_GarageCars,c_GarageArea,v_GarageQual,v_GarageCond,v_PavedDrive,c_WoodDeckSF,c_OpenPorchSF,c_EnclosedPorch,c_3SsnPorch,c_ScreenPorch,c_PoolArea,v_PoolQC,v_Fence,v_MiscFeature,c_MiscVal,v_MoSold,v_YrSold,v_SaleType,v_SaleCondition,tgt_SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,4,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,Ex,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,3,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,Ex,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,4,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,Ex,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.0,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,3,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,Gd,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.0,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
6,7,20,RL,75.0,10084,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Somerst,Norm,Norm,1Fam,1Story,8,5,2004,2005,Gable,CompShg,VinylSd,VinylSd,Stone,186.0,4,TA,PConc,Ex,TA,Av,GLQ,1369,Unf,0,317,1686,GasA,Ex,Y,SBrkr,1694,0,0,1694,1,0,2,0,3,1,Gd,7,Typ,1,Gd,Attchd,2004.0,RFn,2,636,TA,TA,Y,255,57,0,0,0,0,NaN,NaN,NaN,0,8,2007,WD,Normal,307000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1454,1455,20,otro,62.0,7500,Pave,Pave,Reg,Lvl,AllPub,Inside,Gtl,Somerst,Norm,Norm,1Fam,1Story,7,5,2004,2005,Gable,CompShg,VinylSd,VinylSd,None,0.0,4,TA,PConc,Gd,TA,No,GLQ,410,Unf,0,811,1221,GasA,Ex,Y,SBrkr,1221,0,0,1221,1,0,2,0,2,1,Gd,6,Typ,0,NaN,Attchd,2004.0,RFn,2,400,TA,TA,Y,0,113,0,0,0,0,NaN,NaN,NaN,0,10,2009,WD,Normal,185000
1455,1456,60,RL,62.0,7917,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,6,5,1999,2000,Gable,CompShg,VinylSd,VinylSd,None,0.0,3,TA,PConc,Gd,TA,No,Unf,0,Unf,0,953,953,GasA,Ex,Y,SBrkr,953,694,0,1647,0,0,2,1,3,1,TA,7,Typ,1,TA,Attchd,1999.0,RFn,2,460,TA,TA,Y,0,40,0,0,0,0,NaN,NaN,NaN,0,8,2007,WD,Normal,175000
1457,1458,70,RL,66.0,9042,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,9,1941,2006,Gable,CompShg,CemntBd,CmentBd,None,0.0,5,Gd,Stone,TA,Gd,No,GLQ,275,Unf,0,877,1152,GasA,Ex,Y,SBrkr,1188,1152,0,2340,0,0,2,0,4,1,Gd,9,Typ,2,Gd,Attchd,1941.0,RFn,1,252,TA,TA,Y,0,60,0,0,0,0,NaN,GdPrv,Shed,2500,5,2010,WD,Normal,266500
1458,1459,20,RL,68.0,9717,Pave,NaN,Reg,L

In [ ]:
fig = px.histogram(df[["c_LotArea","v_ExterQual"]], x="c_LotArea", color="v_ExterQual", marginal="box")
fig.show()

In [ ]:
histogram(df,"c_BsmtFinSF1",50,"c_BsmtFinSF1")

### Relación de tgt con el resto de variables

In [ ]:
df["tgt_SalePrice"]=df["tgt_SalePrice"].astype(float)
df["c_GrLivArea"]=df["c_GrLivArea"].astype(float)

**MSZoning: Identifies the general zoning classification of the sale.**
- A Agriculture
- C Commercial
- FV Floating Village Residential
- I Industrial
- RH Residential High Density
- RL Residential Low Density
- RP Residential Low Density Park
- RM Residential Medium Density

In [ ]:
from plotly.offline import plot,iplot
pd.options.plotting.backend = "plotly"

In [ ]:
#Superficie habitable sobre el nivel (suelo) pies cuadrados relacionada al precio
df.iplot(kind="scatter", theme="white",x="c_GrLivArea",y="tgt_SalePrice",
            categories="v_MSZoning",xTitle="c_GrLivArea",yTitle="tgt_SalePrice",title="tgt_SalePrice vs c_GrLivArea")

- GarageCars: tamaño del garaje en capacidad de coches
- GarageArea: tamaño del garaje en pies cuadrados
- PoolArea: área de la piscina en pies cuadrados
- PoolQC: calidad de la piscina

**GarageCond: Garage condition**
- Ex Excellent
- Gd Good
- TA Typical/Average
- Fa Fair
- Po Poor
- NA No Garage

In [ ]:
df["c_GarageArea"].value_counts()

0      81
440    49
576    47
240    38
484    34
       ..
320     1
594     1
831     1
878     1
192     1
Name: c_GarageArea, Length: 441, dtype: int64

In [ ]:
df["c_GarageArea"]=df["c_GarageArea"].astype(float)

In [ ]:
df["aux"]="1"

In [ ]:
#Tamaño del garage vs el precio
df.iplot(kind="scatter", theme="white",x="c_GarageArea",y="tgt_SalePrice",categories="aux",colors=["green"],xTitle="c_GarageArea",yTitle="tgt_SalePrice",title="tgt_SalePrice vs c_GarageArea")

In [ ]:
df["v_GarageCond"].value_counts()

TA    1326
Fa      35
Gd       9
Po       7
Ex       2
Name: v_GarageCond, dtype: int64

In [ ]:
df.iplot(kind="scatter", theme="white",x="c_GarageArea",y="tgt_SalePrice",categories="v_GarageCond",xTitle="c_GarageArea",yTitle="tgt_SalePrice",title="tgt_SalePrice vs c_GarageArea")

In [ ]:
df["c_PoolArea"].isnull().sum()

0

In [ ]:
df["c_PoolArea"].describe()

count    1460.000000
mean        2.758904
std        40.177307
min         0.000000
25%         0.000000
50%         0.000000
75%         0.000000
max       738.000000
Name: c_PoolArea, dtype: float64

In [ ]:
df.iplot(kind="scatter", theme="white",x="c_PoolArea",y="tgt_SalePrice",categories="v_PoolQC",xTitle="c_PoolArea",yTitle="tgt_SalePrice",title="tgt_SalePrice vs c_GarageArea")

In [ ]:
#Total de pies cuadrados de área del sótano

In [ ]:
df.iplot(kind="scatter", theme="white",x="c_TotalBsmtSF",y="tgt_SalePrice",categories="aux",xTitle="c_TotalBsmtSF",yTitle="tgt_SalePrice",title="tgt_SalePrice vs c_TotalBsmtSF")

**'TotalBsmtSF' mantienen una relación lineal positiva con 'SalePrice', aumentando en el mismo sentido. En el caso de 'TotalBsmtSF', la pendiente de esta relación es muy acentuada.**

**Overall Qual: (Calificación general: material general y calidad de acabado)**
- 10 Very Excellent
- 9 Excellent
- 8 Very Good
- 7 Good
- 6 Above Average
- 5 Average
- 4 Below Average
- 3 Fair
- 2 Poor
- 1 Very Poor

In [ ]:
df[['tgt_SalePrice', 'v_OverallQual']].pivot(columns='v_OverallQual', values='tgt_SalePrice').iplot(kind='box',theme="white",xTitle="v_OverallQual",yTitle="tgt_SalePrice",title="tgt_SalePrice vs v_OverallQual")

In [ ]:
df[['tgt_SalePrice', 'v_YearBuilt']].pivot(columns='v_YearBuilt', values='tgt_SalePrice').iplot(kind='box',theme="white",xTitle="v_YearBuilt",yTitle="tgt_SalePrice",title="tgt_SalePrice vs v_YearBuilt")

**'OverallQual' y 'YearBuilt' también parecen relacionadas con 'SalePrice' (más fuerte en el primer caso), tal y como se puede observar en los diagramas de cajas**

In [ ]:
df[['tgt_SalePrice', 'v_GarageQual']].pivot(columns='v_GarageQual', values='tgt_SalePrice').iplot(kind='box',theme="white",xTitle="v_GarageQual",yTitle="tgt_SalePrice",title="tgt_SalePrice vs v_GarageQual")

### ANÁLISIS MULTIVARIADO

In [ ]:
#see cufflinks.colors.scales() for available scales

- La coeficiente de correlación puede variar desde -1.00 hasta 1.00.
- La correlación de proporcionalidad directa o positiva se establece con los valores +1.00 y de proporcionalidad inversa o negativa, con -1.00. No existe relación entre las variables cuando el coeficiente es de 0.00.

In [ ]:
df.filter(like="c_").corr()

,c_LotFrontage,c_LotArea,c_MasVnrArea,c_BsmtFinSF1,c_BsmtFinSF2,c_BsmtUnfSF,c_TotalBsmtSF,c_1stFlrSF,c_2ndFlrSF,c_GrLivArea,c_BsmtFullBath,c_HalfBath,c_BedroomAbvGr,c_GarageCars,c_GarageArea,c_WoodDeckSF,c_OpenPorchSF,c_EnclosedPorch,c_3SsnPorch,c_ScreenPorch,c_PoolArea,c_MiscVal
c_LotFrontage,1.000000,0.426095,0.193458,0.233633,0.049900,0.132644,0.392075,0.457181,0.080177,0.402797,0.100949,0.053532,0.263170,0.285691,0.344997,0.088521,0.151972,0.010700,0.070029,0.041383,0.206167,0.003368
c_LotArea,0.426095,1.000000,0.104160,0.214103,0.111170,-0.002618,0.260833,0.299475,0.050986,0.263116,0.158155,0.014259,0.119690,0.154871,0.180403,0.171698,0.084774,-0.018340,0.020423,0.043160,0.077672,0.038068
c_MasVnrArea,0.193458,0.104160,1.000000,0.264736,-0.072319,0.114442,0.363936,0.344501,0.174561,0.390857,0.085310,0.201444,0.102821,0.364204,0.373066,0.159718,0.125703,-0.110204,0.018796,0.061466,0.011723,-0.029815
c_BsmtFinSF1,0.233633,0.214103,0.264736,1.000000,-0.050117,-0.495251,0.522396,0.445863,-0.137079,0.208171,0.649212,0.004262,-0.107355,0.224054,0.296970,0.204306,0.111761,-0.102303,0.026451,0.062021,0.140491,0.003571
c_BsmtFinSF2,0.049900,0.111170,-0.072319,-0.050117,1.000000,-0.209294,0.104810,0.097117,-0.099260,-0.009640,0.158678,-0.032148,-0.015728,-0.038264,-0.018227,0.067898,0.003093,0.036543,-0.029993,0.088871,0.041709,0.004940
c_BsmtUnfSF,0.132644,-0.002618,0.114442,-0.495251,-0.209294,1.000000,0.415360,0.317987,0.004469,0.240257,-0.422900,-0.041118,0.166643,0.214175,0.183303,-0.005316,0.129005,-0.002538,0.020764,-0.012579,-0.035092,-0.023837
c_TotalBsmtSF,0.392075,0.260833,0.363936,0.522396,0.104810,0.415360,1.000000,0.819530,-0.174512,0.454868,0.307351,-0.048804,0.050450,0.434585,0.486665,0.232019,0.247264,-0.095478,0.037384,0.084489,0.126053,-0.018479
c_1stFlrSF,0.457181,0.299475,0.344501,0.445863,0.097117,0.317987,0.819530,1.000000,-0.202646,0.566024,0.244671,-0.119916,0.127401,0.439317,0.489782,0.235459,0.211671,-0.065292,0.056104,0.088758,0.131525,-0.021096
c_2ndFlrSF,0.080177,0.050986,0.174561,-0.137079,-0.099260,0.004469,-0.174512,-0.202646,1.000000,0.687501,-0.169494,0.609707,0.502901,0.183926,0.138347,0.092165,0.208026,0.061989,-0.024358,0.040606,0.081487,0.016197
c_GrLivArea,0.402797,0.263116,0.390857,0.208171,-0.009640,0.240257,0.454868,0.566024,0.687501,1.000000,0.034836,0.415772,0.521270,0.467247,0.468997,0.247433,0.330224,0.009113,0.020643,0.101510,0.170205,-0.002416


In [ ]:
df.filter(like="c_").corr().iplot(kind="heatmap",colorscale="bugn",title="Matriz de Correlación")

El mapa de calor es una forma visual muy útil para para conocer las variables y sus relaciones.
Si existe una correlación significativamente fuerte podría indicar multicolinealidad, es decir, que básicamente ofrecen la misma información.


In [ ]:
list(df.filter(like="c_").columns)+["tgt_SalePrice"]

['c_LotFrontage',
 'c_LotArea',
 'c_MasVnrArea',
 'c_BsmtFinSF1',
 'c_BsmtFinSF2',
 'c_BsmtUnfSF',
 'c_TotalBsmtSF',
 'c_1stFlrSF',
 'c_2ndFlrSF',
 'c_GrLivArea',
 'c_BsmtFullBath',
 'c_HalfBath',
 'c_BedroomAbvGr',
 'c_GarageCars',
 'c_GarageArea',
 'c_WoodDeckSF',
 'c_OpenPorchSF',
 'c_EnclosedPorch',
 'c_3SsnPorch',
 'c_ScreenPorch',
 'c_PoolArea',
 'c_MiscVal',
 'tgt_SalePrice']

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
corr = df[list(df.filter(like="c_").columns)+["tgt_SalePrice"]].corr()
#OverallQual: Material general y calidad de acabado
#c_GrLivArea: pies cuadrados del área habitable sobre el nivel (suelo)
#ExterQual: Calidad del material exterior
corr[['tgt_SalePrice']].sort_values(by = 'tgt_SalePrice',ascending = False).style.background_gradient()

,tgt_SalePrice
tgt_SalePrice,1.000000
c_GrLivArea,0.708624
c_GarageCars,0.640409
c_GarageArea,0.623431
c_TotalBsmtSF,0.613581
c_1stFlrSF,0.605852
c_MasVnrArea,0.477493
c_BsmtFinSF1,0.386420
c_LotFrontage,0.351799
c_WoodDeckSF,0.324413


In [ ]:
#Total de habitaciones sobre rasante (no incluye baños) y Área principal: pies cuadrados del área habitable sobre el nivel (suelo)
#Altamente correalcionada (problema de multicolinealidad)
df[["v_TotRmsAbvGrd","c_GrLivArea"]].corr().iplot(kind="heatmap")

**Diagramas de dispersión entre 'tgt_SalePrice' y sus variables correlacionadas**

In [ ]:
aux=list(corr[['tgt_SalePrice']].sort_values(by = 'tgt_SalePrice',ascending = False)[:4].index)

In [ ]:
aux

['tgt_SalePrice', 'c_GrLivArea', 'c_GarageCars', 'c_GarageArea']

In [ ]:
fig = px.scatter_matrix(df[aux], dimensions=aux,)
fig.show()

### COMPARACIONES

- GrLivArea: pies cuadrados de área habitable sobre el nivel (suelo)

MSZoning: Identifies the general zoning classification of the sale.
A Agriculture
C Commercial
FV Floating Village Residential
I Industrial
RH Residential High Density
RL Residential Low Density
RP Residential Low Density Park
RM Residential Medium Density

In [ ]:
#Superficie habitable sobre el nivel (suelo) pies cuadrados relacionada al precio
df.iplot(kind="scatter", theme="white",x="c_GrLivArea",y="tgt_SalePrice",
            categories="v_MSZoning",xTitle="c_GrLivArea",yTitle="tgt_SalePrice",title="tgt_SalePrice vs c_GrLivArea")

Este diagrama de dispersión muestra un par de cosas interesantes:

Los dos valores más altos de la variable 'c_GrLivArea' resultan extraños.Podría tratarse de terrenos muy degradados, algo que explicaría su bajo precio. Lo que está claro es que estos dos puntos son atípicos.

In [ ]:
df.sort_values(by="c_GrLivArea",ascending=False)[["c_GrLivArea"]]

,c_GrLivArea
1298,5642.0
523,4676.0
1182,4476.0
691,4316.0
1169,3627.0
...,...
528,605.0
29,520.0
916,480.0
1100,438.0


In [ ]:
df=df[df["c_GrLivArea"]<=4476]

In [ ]:
df = px.data.tips()
fig = px.scatter(df, x="total_bill", y="tip", trendline="ols")

In [ ]:
fig.show()

In [ ]:
df

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4
...,...,...,...,...,...,...,...
239,29.03,5.92,Male,No,Sat,Dinner,3
240,27.18,2.00,Female,Yes,Sat,Dinner,2
241,22.67,2.00,Male,Yes,Sat,Dinner,2
242,17.82,1.75,Male,No,Sat,Dinner,2
